# Laboratorium 5 - rekomendacje grafowe

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn gensim==3.8.3`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import random
import numpy as np
import pandas

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold

In [2]:
SCORE_THRESHOLD = 4.0 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen wierzcholkow
NEIGHBOURS_WINDOW = 11 # tylu sasiadow wezmiemy pod uwage w algorytmie Word2Vec (symetrycznie i wliczajac biezacy element)
PATH_LENGTH = 30 # dlugosc pojedynczej losowej sciezki
PATHS_COUNT_PER_NODE = 5 # liczba losowych sciezek zaczynajacych sie w kazdym z wierzcholkow

In [3]:
# wczytujemy oceny uytkownikow

ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
ratings = ratings.where(ratings['rating'] >= SCORE_THRESHOLD).dropna()
# rozszerzamy ID tak, by sie nie powtarzaly
ratings['userId'] = ratings['userId'].apply(lambda x: 'u_' + str(int(x)))
ratings['movieId'] = ratings['movieId'].apply(lambda x: 'm_' + str(int(x)))
ratings

,userId,movieId,rating
0,u_1,m_1,4.0
1,u_1,m_3,4.0
2,u_1,m_6,4.0
3,u_1,m_47,5.0
4,u_1,m_50,5.0
...,...,...,...
100830,u_610,m_166528,4.0
100831,u_610,m_166534,4.0
100832,u_610,m_168248,5.0
100833,u_610,m_168250,5.0


In [4]:
# wczytujemy gatunki filmow

movies = pandas.read_csv('ml-latest-small/movies.csv').drop(columns=['title'])
movies['movieId'] = movies['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies_to_genres = movies.explode('genres')
movies_to_genres['genres'] = movies_to_genres['genres'].apply(lambda x: 'g_' + x.lower())
movies_to_genres = movies_to_genres.rename(columns = {'genres': 'genre'})
movies_to_genres

,movieId,genre
0,m_1,g_adventure
0,m_1,g_animation
0,m_1,g_children
0,m_1,g_comedy
0,m_1,g_fantasy
...,...,...
9738,m_193583,g_fantasy
9739,m_193585,g_drama
9740,m_193587,g_action
9740,m_193587,g_animation


In [5]:
users = ratings['userId'].unique()
movies = ratings['movieId'].unique()
genres = movies_to_genres['genre'].unique()

## Część 2. - spacer po grafie

In [7]:
# generujemy losowe sciezki w grafie
#   krawedzie reprezentowane sa w dwoch macierzach - ratings i movies
#   w wersji podstawowej wszystkie krawedzie traktujemy jako niewazone i nieskierowane
#   mozliwe ulepszenia:
#    - rozwazenie krawedzi skierowanych
#    - uwzglednienie wag krawedzi (ocen uzytkownikow)
#    - jakas forma normalizacji - obnizenia wag wierzcholkow o wysokich stopniach
#    - Node2Vec - parametry P i Q
# wynikiem powinna byc lista list - kazda z tych list zawiera kolejne ID wierzcholkow na sciezce

def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    movies_no = len(movies)
    for idx, movie in enumerate(movies):
        percent = idx/movies_no
        bar = '{progress:-<100}'.format(progress='■'*math.floor(percent*100))
        print(f'\r{movie} |{bar}| {"\r", (percent*100):.2f}%', end = "")
        
        for i in range(paths_per_node):
            nodes = []
            # wybieramy losowo pierwszy wierzchołek spośród wszystkich klas
            nodes.append(movie)
            for j in range(path_length-1):
                last_node = nodes[-1]
                # na podstawie ostatniego nodea wybieramy krawędź
                if last_node[0] == 'u':
                    new_node = ratings.loc[ratings['userId'] == last_node].sample()['movieId']
                elif last_node[0] == 'm':
                    # movie ma krawędzie z gatunkami i userami, losujemy dla której klasy wybrać krawędź
                    # film może nie być oceniony, więc w tym przypadku bierzemy gatunek
                    new_node = ratings.loc[ratings['movieId'] == last_node]
                    if random.random() < 0.5 and new_node.size > 0:
                        new_node = new_node.sample()['userId']
                    else:
                        new_node = movies_to_genres.loc[movies_to_genres['movieId'] == last_node].sample()['genre']
                elif last_node[0] == 'g':
                    new_node = movies_to_genres.loc[movies_to_genres['genre'] == last_node].sample()['movieId']
                    
                nodes.append(new_node.iloc[0])
        
            paths.append(nodes)
            
    print('Done', end = "\r")
    
    return paths
    
walks = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)

## Część 3. - obliczenie osadzeń

In [8]:
# trenujemy model
#   zauwaz, ze wszystkie trzy rodzaje wierzcholkow beda reprezentowane tak samo, w tej samej przestrzeni

model = Word2Vec(sentences=walks, vector_size=VECTOR_SIZE, window=NEIGHBOURS_WINDOW, min_count=1, workers=4)
embeddings = model.wv
[u_1, m_1, g_1, m_71231, u_27]
[10, 0.1234, 42.322]

## Część 4. - rekomendacje i zastosowania

In [9]:
PULP_FICTION = 'm_296'
TOY_STORY = 'm_1'
PLANET_OF_THE_APES = 'm_2529'

In [10]:
# wyszukajmy K najpodobniejszych filmów do danego
# porownaj wyniki dla odleglosci euklidesowej i cosinuswej, np. na trzech powyzszych filmach

def euclidian_distance(i, j):
    distance = np.subtract(i, j)
    distance = np.square(distance)
    distance = np.sum(distance)
    distance = math.sqrt(distance)
    
    return distance

def cosine_distance(i, j):
    dot_prod = np.multiply(i, j)
    dot_prod = np.sum(dot_prod)
    
    magn_i = np.square(i)
    magn_i = np.sum(magn_i)
    magn_i = math.sqrt(magn_i)
    
    magn_j = np.square(j)
    magn_j = np.sum(magn_j)
    magn_j = math.sqrt(magn_j)
    
    return dot_prod/(magn_i*magn_j)
    

def k_most_similar_movies(movie_id, K, embeddings, distance_fun):
    # ...
    # sortujemy movies(z wykluczeniem movie_id) po odległości embeddingów
    k_most_similar = [movie for movie in movies if movie != movie_id]
    k_most_similar.sort(key=(lambda x: distance_fun(embeddings[movie_id], embeddings[x])))
    return k_most_similar[:K]

print(k_most_similar_movies(PULP_FICTION, 5, embeddings, cosine_distance))
print(k_most_similar_movies(TOY_STORY, 5, embeddings, cosine_distance))
print(k_most_similar_movies(PLANET_OF_THE_APES, 5, embeddings, cosine_distance))

['m_8372', 'm_4697', 'm_136341', 'm_40339', 'm_104875']
['m_130482', 'm_80862', 'm_61257', 'm_4957', 'm_148982']
['m_1703', 'm_105746', 'm_184257', 'm_131724', 'm_131013']


In [11]:
# wyszukajmy k filmow najblizszych uzytkownikowi
# wykorzystaj funkcje z poprzedniej komorki

def k_best_movies_for_user(user_id, K, embeddings, distance_fun):
    return k_most_similar_movies(user_id, K, embeddings, distance_fun)

k_best_movies_for_user('u_1', 5, embeddings, cosine_distance)

['m_8577', 'm_61257', 'm_104272', 'm_80862', 'm_6869']

In [12]:
# sprobujmy czegos bardziej skomplikowanego
#   znajdz ulubiony gatunek filmowy uzytkownika
#   a nastepnie zaproponuj K filmow z tego gatunku - ale nie tych najblizszych uzytkownikowi
#   (zaproponuj, w jaki sposob dobrac filmy interesujace, ale nie z najblizszego otoczenia)

def k_from_favourite_genre(user_id, K, embeddings, distance_fun):
    # ...
    k_most_similar = [genre for genre in genres]
    k_most_similar.sort(key=(lambda x: distance_fun(embeddings[user_id], embeddings[x])))
    favourite_genre = k_most_similar[0]
    
    return k_most_similar_movies(favourite_genre, K, embeddings, distance_fun)

k_from_favourite_genre('u_1', 5, embeddings, cosine_distance)

['m_349', 'm_367', 'm_212', 'm_22', 'm_168']

In [13]:
# Na koniec najbardziej skomplikowany algorytm - odpowiednik "Radia utworu" w Spotify.
#   Zaczynamy od jednego filmu, a nastepnie wyznaczamy kolejne, wedrujac po przestrzeni, w ktorej wszystkie elementy sa osadzone.
#   Zaproponuj, jak zdefiniowac podzbior filmow, z ktorych bedziemy wybierac (np. filmy odlegle o min. a i max. b od danego)
#   oraz jak generowac kolejny skok (tak, zeby seria rekomendacji nie byla zbyt monotonna, ale rownoczesnie zgodna z gustem uzytkownika)

def radio_distance(u, m1, m2, min_dist, max_dist, movie_bias = 0.7, distance_fun=cosine_distance):
    distance = movie_bias*distance_fun(m1, m2)+(1-movie_bias)*distance_fun(m2, u)
    
    if distance<min_dist or distance>max_dist:
        distance = math.inf
        
    return distance


def get_playlist(start_movie_id, user_id, K, embeddings):
    # ...
    playlist = k_most_similar_movies(start_movie_id, K, embeddings, lambda i, j: radio_distance(user_id, i, j, 0.1, 100))    
      
    
    return playlist

In [14]:
get_playlist('m_1', 'u_1', 10, embeddings)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('float32'), dtype('<U3')) -> None